# NUSTAR SCIENTIFIC PHASE-RESOLVED PRODUCTS EXTRACTION

### Initialize  NustarObservation object of particular ObsID
choose the period, whether to shift the zero-phase time, etc

In [1]:
import sys; sys.path.append('/sj0243/')

In [4]:
from python_for_nustar.nu_core import np, plt, os, glob, scan_phase_resolved_products, reduce_list, fits
from python_for_nustar.nu_class import NustarObservation
from python_for_nustar.pipeline.sj0243_init import Nu_path, ObsList_bright, ObsList, periods_val

ObsID = ObsList[2] #change here index to iterate over observations
shift = 0  #0 or 1. shifting the phase-zero time to half the phase bin width
period = periods_val[ObsID]
nu_obs = NustarObservation(ObsID, nu_path=Nu_path)


###
Observation 90302319006 loaded successfully


### GTI creation and extracting phase-resolved nustar products

In [10]:
prodpath_ave = 'spe_and_lc' #from light curves in this folder we will make GTIs

if shift == 0:
    prodpath_ph_res = 'phase_resolved' 
    nu_obs.make_gti_from_lc(prodpath=prodpath_ave, mode='AB', period=period, phase_bins=10, half_shift=0, outfolder = 'gtis/' )
    for mode in ['A','B']:
        nu_obs.phase_resolved_spectra(gtipath=prodpath_ave+'/gtis', mode=mode, folder='phase_resolved')
elif shift == 1:
    prodpath_ph_res = 'phase_resolved_shift_1' 
    nu_obs.make_gti_from_lc(prodpath=prodpath_ave, mode='AB', period=period, phase_bins=10, half_shift=1, outfolder = 'gtis_shift_1/' )
    for mode in ['A','B']:
        nu_obs.phase_resolved_spectra(gtipath=prodpath_ave+'/gtis_shift_1', mode=mode, folder='phase_resolved_shift_1')

os.chdir(prodpath_ph_res)
open_dir_in_term()

['gtis_shift_1/gti_2A.fits', 'gtis_shift_1/gti_11B.fits', 'gtis_shift_1/gti_13B.fits', 'gtis_shift_1/gti_4A.fits', 'gtis_shift_1/gti_8B.fits', 'gtis_shift_1/gti_15B.fits', 'gtis_shift_1/gti_6A.fits', 'gtis_shift_1/gti_12B.fits', 'gtis_shift_1/gti_1A.fits', 'gtis_shift_1/gti_3A.fits', 'gtis_shift_1/gti_10B.fits', 'gtis_shift_1/gti_14B.fits', 'gtis_shift_1/gti_7A.fits', 'gtis_shift_1/gti_5A.fits', 'gtis_shift_1/gti_9B.fits', 'gtis_shift_1/gti_16B.fits', 'gtis_shift_1/gti_5B.fits', 'gtis_shift_1/gti_9A.fits', 'gtis_shift_1/gti_16A.fits', 'gtis_shift_1/gti_14A.fits', 'gtis_shift_1/gti_7B.fits', 'gtis_shift_1/gti_3B.fits', 'gtis_shift_1/gti_10A.fits', 'gtis_shift_1/gti_12A.fits', 'gtis_shift_1/gti_1B.fits', 'gtis_shift_1/gti_15A.fits', 'gtis_shift_1/gti_6B.fits', 'gtis_shift_1/gti_4B.fits', 'gtis_shift_1/gti_8A.fits', 'gtis_shift_1/gti_13A.fits', 'gtis_shift_1/gti_2B.fits', 'gtis_shift_1/gti_11A.fits']
GTIs have been created for module(s) AB with period 9.8234 and 10 bins, phase bin number 

 This little fucntion make a list of products filenames, such as the names of spectral files of each phase, light curves of each phase, etc

In [11]:

lclist, lclist_bary, lclistorb_corr, spelist, propname_per_bin = scan_phase_resolved_products(prodpath=prodpath_ph_res, n_phase = 10)


### sum all thee lightcurves, barycenter them, and correct for orbital motion


In [12]:

for pair in lclist:
    binnum = pair[0].split('_')[2][3:-1]
    nu_obs.lcmath(infiles=pair, outfile=f'phase_resolved_bin{binnum}AB_sr.lc', prodpath=prodpath_ph_res, cmd_name='lcmath_orig', rewrite=False)

for pair in propname_per_bin(['AB'], postfix = 'lc'):
    nu_obs.barycorr(infiles=pair, prodpath=prodpath_ph_res, barytime='no', rewrite=False)


Creating command command: 
        lcmath         infile=phase_resolved_bin1A_sr.lc         bgfile = phase_resolved_bin1B_sr.lc         outfile = phase_resolved_bin1AB_sr.lc         multi = 0.5         multb = 0.5         addsubr = yes
        
Writing to file:  lcmath_orig
Creating command command: 
        lcmath         infile=phase_resolved_bin2A_sr.lc         bgfile = phase_resolved_bin2B_sr.lc         outfile = phase_resolved_bin2AB_sr.lc         multi = 0.5         multb = 0.5         addsubr = yes
        
Writing to file:  lcmath_orig
Creating command command: 
        lcmath         infile=phase_resolved_bin3A_sr.lc         bgfile = phase_resolved_bin3B_sr.lc         outfile = phase_resolved_bin3AB_sr.lc         multi = 0.5         multb = 0.5         addsubr = yes
        
Writing to file:  lcmath_orig
Creating command command: 
        lcmath         infile=phase_resolved_bin4A_sr.lc         bgfile = phase_resolved_bin4B_sr.lc         outfile = phase_resolved_bin4AB_sr.lc  

In [13]:

for pair in propname_per_bin(['AB'], postfix = 'lc_bary'):
    nu_obs.orb_correction_lc(
    filename=pair[0], prodpath=prodpath_ph_res)


no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes
no orbital parameters given, copying the original file without changes


### make pulse profiles of each phase bin. Needed to check if the phases are correctly set


In [14]:
for pair in propname_per_bin(['AB'], postfix = 'lc_bary_orb_corr'):
    nu_obs.make_efold(filename = pair[0], prodpath = prodpath_ph_res, period = period, cmd_name='efold', nphase='128', rewrite=False)

Creating command command: 
            efold             nser = 1             norm = 0             cfile1 = phase_resolved_bin1AB_sr.lc_bary_orb_corr             dper = 9.8234             nphase = 128             nbint = INDEF             nintfm = INDEF             outfileroot = 'default'             window = '-'             sepoch = 0             plot = no              outfile = phase_resolved_bin1AB_sr.lc_bary_orb_corr_nphase_128.efold
Writing to file:  efold
Creating command command: 
            efold             nser = 1             norm = 0             cfile1 = phase_resolved_bin2AB_sr.lc_bary_orb_corr             dper = 9.8234             nphase = 128             nbint = INDEF             nintfm = INDEF             outfileroot = 'default'             window = '-'             sepoch = 0             plot = no              outfile = phase_resolved_bin2AB_sr.lc_bary_orb_corr_nphase_128.efold
Writing to file:  efold
Creating command command: 
            efold             nser = 1   

### check if thee pulse profile of phases are consequal and do not overlap

In [15]:
efolds = propname_per_bin(modes = ['AB'], postfix='lc_bary_orb_corr_nphase_128.efold')
nu_obs.check_efold_of_bins(efolds_files=reduce_list(efolds),prodpath=prodpath_ph_res, fiducial = '../spe_and_lc/spe_and_lcAB_sr.lc_bary_orb_corr_nphase_128.efold')

working with phase_resolved_bin1AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin2AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin3AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin4AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin5AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin6AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin7AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin8AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin9AB_sr.lc_bary_orb_corr_nphase_128.efold
working with phase_resolved_bin10AB_sr.lc_bary_orb_corr_nphase_128.efold
/Users/sdbykov/work/my_packages/nustar_pipeline/nustar_pipeline/nustar_obs_class.py:560: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  fig.tight_layout()


In [ ]:
#light_curves = propname_per_bin(modes = ['AB'], postfix='lc_bary_orb_corr')
#nu_obs.check_lightcurve_of_bins(lc_files=reduce_list(light_curves),prodpath=prodpath_ph_res, fiducial = '../spe_and_lc/spe_and_lcAB_sr.lc_bary_orb_corr')


### group spectral files

In [16]:
for pair in propname_per_bin(['A','B'], postfix = 'pha'):
    nu_obs.grppha(infiles=pair, prodpath=prodpath_ph_res, group_min=25)


Creating command command: grppha infile="phase_resolved_bin1A_sr.pha" outfile="phase_resolved_bin1A_sr.pi"  comm="group min 25 & exit" clobber=yes
Writing to file:  grppha
Creating command command: grppha infile="phase_resolved_bin1B_sr.pha" outfile="phase_resolved_bin1B_sr.pi"  comm="group min 25 & exit" clobber=yes
Writing to file:  grppha
Creating command command: grppha infile="phase_resolved_bin2A_sr.pha" outfile="phase_resolved_bin2A_sr.pi"  comm="group min 25 & exit" clobber=yes
Writing to file:  grppha
Creating command command: grppha infile="phase_resolved_bin2B_sr.pha" outfile="phase_resolved_bin2B_sr.pi"  comm="group min 25 & exit" clobber=yes
Writing to file:  grppha
Creating command command: grppha infile="phase_resolved_bin3A_sr.pha" outfile="phase_resolved_bin3A_sr.pi"  comm="group min 25 & exit" clobber=yes
Writing to file:  grppha
Creating command command: grppha infile="phase_resolved_bin3B_sr.pha" outfile="phase_resolved_bin3B_sr.pi"  comm="group min 25 & exit" clobb

# plot spectral ratio between different phase bins (no fitting)

In [35]:
from pyxspeclib.definitions import AllData, Plot
from nustar_pipeline.definitions import ratio_error, PIfromE
def make_grppha_and_wd(folder, pha_files, NCHAN_1='5', NCHAN_2 = '15',
                       en_lo='4.', en_hi='30.', en_split = 15,
                       labels=None):
    os.chdir(folder)

    enarr = []
    enerrarr = []
    dataarr = []
    dataerrarr = []

    if labels is None:
        labels = pha_files

    for infile in pha_files:
        stemname = infile.split('.')[0:-1]
        stemname = '.'.join(stemname)
        outfile = stemname+'.pi_linbin'
        split_chan = int(PIfromE(en_split))
        # grppha = f'''grppha infile="{infile}" outfile="{outfile}"  comm="group 1 4096 {NCHAN} & exit & exit" clobber=yes'''
        grppha = f'''grppha infile="{infile}" outfile="{outfile}"  comm="group 1 {split_chan} {NCHAN_1} {split_chan+1} 4096 {NCHAN_2} & exit" clobber=yes'''
        print(grppha)
        os.system(grppha)

        AllData.clear()
        AllData(f"1:1 {outfile}")
        AllData.ignore(f'**-{en_lo} {en_hi}-**')
        # AllData.ignore("bad")
        Plot.device = '/null'
        Plot('da')
        Plot.xAxis = 'keV'
        en = Plot.x()
        data = Plot.y()
        en_err = Plot.xErr()
        data_err = Plot.yErr()

        enarr.append(en)
        dataarr.append(data)
        dataerrarr.append(data_err)
        enerrarr.append(en_err)

    enarr = np.array(enarr)
    dataarr = np.array(dataarr)
    dataerrarr = np.array(dataerrarr)
    enerrarr = np.array(enerrarr)

    elo_ins = 5.5
    ehi_ins = 8.5
    fig, ax = plt.subplots(figsize=(4, 8))
    #ax_zoom = ax.inset_axes([0.05, 0.70, 0.15, 0.35])    # type: ignore
    # ax_zoom = ax.inset_axes([0.05, 0.25, 0.45, 0.65])    # type: ignore
    #ax_zoom = ax.inset_axes([0.05, 0.25, 0.001, 0.001])    # type: ignore
    # ax.indicate_inset_zoom(ax_zoom)

    for i, (d, e) in enumerate(zip(dataarr[1:], dataerrarr[1:]), 1):
        rat, rat_err = ratio_error(d, dataarr[0], e, dataerrarr[0])

        ax.step(enarr[0], rat, 'o-',
                label=f"{labels[i]}/{labels[0]}", where='mid', alpha=0.7)
        color = ax.get_lines()[-1].get_color()
        ax.errorbar(enarr[0], rat, rat_err, enerrarr[0],
                    ecolor=color, fmt='none', alpha=0.4)

        idx = (enarr[0] < ehi_ins) & (enarr[0] > elo_ins)
        #ax_zoom.step(enarr[0][idx], rat[idx],
        #             'o-', color=color, where='mid', alpha=0.7)
        #ax_zoom.errorbar(enarr[0][idx], rat[idx],
        #                 rat_err[idx], ecolor=color, fmt='none', alpha=0.4)

        # ax_zoom.step(enarr[0][idx], rat[idx]/rat[idx][0],
        #              'o-', color=color, where='mid', alpha=0.7)
        # ax_zoom.errorbar(enarr[0][idx], rat[idx]/rat[idx][0],
        #                  rat_err[idx]/rat[idx][0], ecolor=color, fmt='none', alpha=0.4)

    for axt in [ax]:#, ax_zoom]:
        axt.set_xscale('log')
        axt.axvline(6.4, color='k', ls='--', lw=0.5)
        axt.axvline(6.7, color='k', ls='--', lw=0.5)
        axt.axvline(7.1, color='k', ls='--', lw=0.5)
    #ax_zoom.set_xlim(elo_ins, ehi_ins)
    #ax_zoom.set_yscale('log')
    #ax.set_ylim(0.5, 2)
    #ax.set_xlim(0.1, float(en_hi))
    ax.set_yscale('log')

    ax.legend(loc='lower left')
    #ax.legend(bbox_to_anchor=(1, 0.5), loc='center left', fontsize=8)

make_grppha_and_wd(folder='/Users/sdbykov/work/xray_pulsars/sj0243_nu/results/out90302319004/products/phase_resolved_shift_1/', pha_files=[
                    'phase_resolved_bin2A_sr.pha', 'phase_resolved_bin10A_sr.pha', 'phase_resolved_bin4A_sr.pha', 'phase_resolved_bin7A_sr.pha', 'phase_resolved_bin3A_sr.pha'], NCHAN_1=5, NCHAN_2=25, en_split=10.)

# make_grppha_and_wd(folder='/Users/sdbykov/work/xray_pulsars/sj0243_nu/results/out90302319004/products/phase_resolved_shift_1/', pha_files=[
#                     'phase_resolved_bin2B_sr.pha', 'phase_resolved_bin10B_sr.pha', 'phase_resolved_bin4B_sr.pha', 'phase_resolved_bin7B_sr.pha', 'phase_resolved_bin3B_sr.pha'], NCHAN_1=5, NCHAN_2=25, en_split=10.)


# make_grppha_and_wd(folder='/Users/sdbykov/work/xray_pulsars/sj0243_nu/results/out90302319006/products/phase_resolved/', pha_files=[
#                    'phase_resolved_bin1A_sr.pha', 'phase_resolved_bin2A_sr.pha', 'phase_resolved_bin3A_sr.pha', 'phase_resolved_bin4A_sr.pha',  'phase_resolved_bin5A_sr.pha', 'phase_resolved_bin7A_sr.pha', 'phase_resolved_bin9A_sr.pha'], NCHAN=5)


# make_grppha_and_wd(folder='/Users/sdbykov/work/xray_pulsars/sj0243_nu/results/out90401334002/products/phase_resolved', pha_files=['phase_resolved_bin9A_sr.pha', 'phase_resolved_bin2A_sr.pha', 'phase_resolved_bin3A_sr.pha', 'phase_resolved_bin4A_sr.pha', 'phase_resolved_bin7A_sr.pha', 'phase_resolved_bin1A_sr.pha'], NCHAN='40', en_hi = '79.')

grppha infile="phase_resolved_bin2A_sr.pha" outfile="phase_resolved_bin2A_sr.pi_linbin"  comm="group 1 210 5 211 4096 25 & exit" clobber=yes
grppha infile="phase_resolved_bin10A_sr.pha" outfile="phase_resolved_bin10A_sr.pi_linbin"  comm="group 1 210 5 211 4096 25 & exit" clobber=yes
grppha infile="phase_resolved_bin4A_sr.pha" outfile="phase_resolved_bin4A_sr.pi_linbin"  comm="group 1 210 5 211 4096 25 & exit" clobber=yes
grppha infile="phase_resolved_bin7A_sr.pha" outfile="phase_resolved_bin7A_sr.pi_linbin"  comm="group 1 210 5 211 4096 25 & exit" clobber=yes
grppha infile="phase_resolved_bin3A_sr.pha" outfile="phase_resolved_bin3A_sr.pi_linbin"  comm="group 1 210 5 211 4096 25 & exit" clobber=yes
grppha infile="phase_resolved_bin2B_sr.pha" outfile="phase_resolved_bin2B_sr.pi_linbin"  comm="group 1 210 5 211 4096 25 & exit" clobber=yes
grppha infile="phase_resolved_bin10B_sr.pha" outfile="phase_resolved_bin10B_sr.pi_linbin"  comm="group 1 210 5 211 4096 25 & exit" clobber=yes
grppha in